# Generic Resume Parser

**Goal:** Given a resume (PDF or Word), extract a structured JSON object:

## Outline

1. **Setup & Requirements** – optional installs, environment configuration  
2. **Project Structure & Settings** – skill list, file handling options  
3. **Text Extraction** – robust loaders for PDF/DOCX/TXT  
4. **Gemini Parser** – JSON‑only extraction with strict schema  
5. **Fallback Parser** – rules‑based extraction (email, name, skills)  
6. **Unified Pipeline** – `parse_resume(file_path)` chooses the best available method  



## 1) Setup & Requirements
- Install dependencies (if not already available)



**Environment variable :**  
Set api key as `GEMINI_API_KEY` in your local .env file.

In [1]:
import sys, os
from dotenv import load_dotenv
import google.generativeai as genai
import spacy

print("Python:", sys.version)
print("Interpreter:", sys.executable)

load_dotenv()  # loads .env from project root
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
assert GEMINI_API_KEY, "Set GEMINI_API_KEY in your .env"

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
print("Gemini OK →", model.generate_content("Say hi in 3 words.").text.strip())

nlp = spacy.load("en_core_web_sm")
print("spaCy OK →", nlp.pipe_names)


Python: 3.11.9 (v3.11.9:de54cf5be3, Apr  2 2024, 07:12:50) [Clang 13.0.0 (clang-1300.0.29.30)]
Interpreter: /Users/christinewei/Documents/resume-parser/.venv/bin/python
Gemini OK → Hello, friend!
spaCy OK → ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [ ]:
## Uncoment the lines below to install necessary libraries if you use colab
# !pip install c google-generativeai python-dotenv
# !pip install spacy
# !python -m spacy download en_core_web_sm


In [2]:
import os, json, re
import fitz  # PyMuPDF
import docx
import pandas as pd
import numpy as np
import spacy

### 2) Project Structure & Settings

We keep a **small default skill list** to ensure the fallback works out of the box.  Here we use the esco_skills.csv dataset from Kaggle: https://www.kaggle.com/datasets/thenoob69/esco-skills

Optionally, you can **extend** it by pointing to your own CSV or TXT file.

In [3]:
# Build a default/general skills list set for fallback skill parse matching check

def build_skill_map_from_csv(csv_path='esco_skills.csv'):
    """
    Loads skills from the Kaggle ESCO CSV and creates a mapping from any
    skill variation (alternative or primary) to its canonical name.

    Args:
        csv_path (str): The path to the Kaggle skills CSV file.

    Returns:
        dict: A dictionary mapping every possible skill alias (in lowercase)
              to its canonical, original-cased skill name. Returns an empty
              dict if the file cannot be read.
    """
    try:
        df = pd.read_csv(csv_path)
        print(f"Successfully loaded {csv_path} with {len(df)} rows.")
    except FileNotFoundError:
        print(f"Error: The file was not found at {csv_path}.")
        print("Please download the Kaggle ESCO skills CSV and place it in the correct directory.")
        return {}

    # Create the mapping dictionary
    skill_map = {}

    # Drop rows where the primary skill label is missing, as they are unusable
    df.dropna(subset=['label_cleaned'], inplace=True)

    # Replace NaN in 'altLabels' with an empty string to prevent errors
    df['altLabels'].fillna('', inplace=True)

    for index, row in df.iterrows():
        # Get the canonical skill name (e.g., "Manage Musical Staff")
        canonical_skill = row['label_cleaned'].strip()

        # Map the lowercase version of the canonical skill to itself
        skill_map[canonical_skill.lower()] = canonical_skill

        # Process the alternative labels
        alt_labels_str = row['altLabels']

        # The altLabels are often a long string; we can split them if a clear delimiter exists
        # or treat common phrases. For robustness, we will treat them as space-separated words
        # and also add the full strings. Note: This part might need refinement based on the exact
        # format of the altLabels string. 
        alt_labels_list = alt_labels_str.strip().split('\n')

        for alt_label in alt_labels_list:
            alt_label = alt_label.strip()
            if alt_label:
                # Map the lowercase version of the alias to the canonical skill
                skill_map[alt_label.lower()] = canonical_skill

    print(f"Built a skill map with {len(skill_map)} total variations.")
    return skill_map

# --- Example Usage ---
# Make sure to provide the correct path to your downloaded CSV file.
# SKILL_MAP = build_skill_map_from_csv('/content/skills.csv')
# print(SKILL_MAP)

### 3) Text Extraction

Robust file loader that supports **PDF**, **DOCX**, and **TXT**.  
- Prefers `PyMuPDF` (fast/accurate) if installed.  
- DOCX via `python-docx`.  


In [4]:
def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file."""
    try:
        doc = fitz.open(file_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error reading PDF {file_path}: {e}")
        return None

def extract_text_from_docx(file_path):
    """Extracts text from a .docx file."""
    try:
        doc = docx.Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
        return text
    except Exception as e:
        print(f"Error reading DOCX {file_path}: {e}")
        return None

def extract_text(file_path):
    """
    Detects the file type and uses the appropriate function to extract text.
    Returns the extracted text as a string.
    """
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension == '.docx':
        return extract_text_from_docx(file_path)
    else:
        print(f"Unsupported file format: {file_extension}")
        return None

# --- Example Usage ---
# Make sure to provide the correct path to your downloaded sample PDF or Word file for testing
# sample_file_path = '~/sample_resume7.docx'
# if os.path.exists(sample_pdf_path):
#     extracted_content = extract_text(sample_file_path)
#     print("--- Extracted Content ---")
#     print(extracted_content)


### 4) Gemini Parser (LLM‑first)

We ask Gemini to output **strict JSON** with exactly the keys: `name`, `email`, `skills`.
If the model is unavailable or no key is set, we **skip** this path and let the unified pipeline use the fallback.

In [5]:
def parse_resume_with_genai(resume_text):
    """
    Sends resume text to the GenAI model and asks for structured data extraction.
    """
    if not resume_text:
        return None

    # This prompt is the key to the success of the parser.
    # It clearly defines the task, the input, and the desired output format.
    prompt = f"""
    You are an expert resume parser. Your task is to analyze the provided resume text and extract the following information: the candidate's full name, their email address, and a list of their skills.

    The output MUST be a valid JSON object with the following structure:
    {{
      "name": "...",
      "email": "...",
      "skills": ["...", "...", "..."]
    }}

    Do not include any explanations, introductions, or additional text outside of the JSON object.

    Here is the resume text:
    ---
    {resume_text}
    ---
    """

    try:
        response = model.generate_content(prompt)

        # Clean the response to ensure it's valid JSON
        json_response_text = response.text.strip().replace('```json', '').replace('```', '')

        # Parse the JSON string into a Python dictionary
        return json.loads(json_response_text)
        # return response

    except Exception as e:
        print(f"An error occurred during GenAI parsing: {e}")
        print(f"Raw response was: {response.text}")
        return None



### 5) Fallback Parser (Regex + Heuristics)

- **Email** via robust regex  
- **Name** from early lines (2–4 words, title case, avoid section headers)  
- **Skills** via intersection against a skill list (configurable)


**Name Extraction:**

Extracts candidate names using a tiered approach:

1) spaCy NER , 2) regex near email, and 3) regex from top of text as fallback.


In [6]:
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print("Spacy model not found. Please run: !python -m spacy download en_core_web_sm")
    nlp = None

def clean_extracted_name(name_text):
  """
  Cleans the extracted name by taking only the first line.
  This solves the issue of capturing a name plus a job title on the next line.
  """
  if name_text is None:
      return None

  # Split the text by newline characters and take the first element
  return name_text.splitlines()[0].strip()

def extract_name_with_ner(text, nlp_model):
    """
    Uses spaCy's Named Entity Recognition (NER) to find person names.
    This is the most reliable method.
    """
    if not nlp_model:
        return None

    doc = nlp_model(text)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            # Check if the name seems plausible (e.g., more than one word)
            if len(ent.text.strip().split()) >= 2:
                return clean_extracted_name(ent.text)
    return None

def extract_name_near_email(text, email):
    """
    Finds a name-like pattern on the same line as or line above the email.
    """
    if not email:
        return None

    # A more flexible regex for names, allowing for initials and hyphens
    name_regex = r"([A-Z][a-z'-]+(?:\s+[A-Z][a-z'-]+|\s+[A-Z]\.)+)"
    lines = text.splitlines()

    for i, line in enumerate(lines):
        if email in line:
            # 1. Check the same line as the email
            match = re.search(name_regex, line)
            if match:
                # Ensure we didn't just match part of an email or URL
                if '@' not in match.group(0):
                    return clean_extracted_name(match.group(0))

            # 2. If not found, check the line directly above (if it exists)
            if i > 0:
                match = re.search(name_regex, lines[i-1])
                if match:
                    return clean_extracted_name(match.group(0))
    return None

def extract_name_from_top(text):
    """
    Cleans the top of the resume and uses regex to find the most likely name.
    This is the final fallback method.
    """
    # Look at the first 300 characters
    text_top = text[:300]

    # More flexible regex allowing for middle initials, different spacing, etc.
    name_regex = r"([A-Z][a-z'-]+(?:\s+[A-Z][a-z'-]+|\s+[A-Z]\.)+)"

    match = re.search(name_regex, text_top)
    if match:
        return clean_extracted_name(match.group(0))
    return None



Fallback Parser

In [7]:
def parse_resume_with_rules(resume_text, skill_map, nlp_model):
    """
    Parses resume text using a multi-layered, rule-based approach.

    Args:
        resume_text (str): The raw text of the resume.
        skill_map (dict): A mapping of skill variations to canonical names.
        nlp_model: The loaded spaCy model for NER.

    Returns:
        dict: A dictionary containing the extracted information.
    """
    if not resume_text:
        return None

    # --- Email Extraction (Do this first as it can help with name extraction) ---
    email = None
    email_match = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', resume_text)
    if email_match:
        email = email_match.group(0)

    # --- Name Extraction (Multi-layered approach) ---
    name = None
    # 1. Try NER first - it's the most intelligent
    name = extract_name_with_ner(resume_text, nlp_model)

    # 2. If NER fails, try finding the name near the email
    if not name:
        name = extract_name_near_email(resume_text, email)

    # 3. As a last resort, use the improved top-of-document search
    if not name:
        name = extract_name_from_top(resume_text)

    # --- Skill Extraction (Using the advanced SKILL_MAP) ---
    found_canonical_skills = set()
    resume_text_lower = resume_text.lower()
    if skill_map:
        for skill_alias, canonical_skill in skill_map.items():
            if re.search(r'\b' + re.escape(skill_alias) + r'\b', resume_text_lower):
                found_canonical_skills.add(canonical_skill)

    # --- Assemble final JSON object ---
    parsed_data = {
        "name": name,
        "email": email,
        "skills": sorted(list(found_canonical_skills))
    }

    return parsed_data


# --- FINAL EXECUTION EXAMPLE ---
# 1. Load dependencies at the start
# SKILL_MAP = build_skill_map_from_csv('skills.csv')
# nlp = spacy.load('en_core_web_sm')



### 6) Unified Pipeline

`parse_resume(file_path)` tries **Gemini** first (if available), otherwise falls back to the rules‑based parser.


In [8]:
def process_resume(file_path, skill_map):
    """
    Main function to process a resume file.
    It extracts text, tries the GenAI parser, and uses a rule-based
    fallback with an advanced skill map if the GenAI parser fails.
    """
    print(f"\n--- Processing {os.path.basename(file_path)} ---")

    text = extract_text(file_path)
    if not text:
        print("Could not extract text. Aborting.")
        return None, "Error"

    print("Attempting to parse with GenAI...")
    parsed_data = parse_resume_with_genai(text)

    if parsed_data is None:
        print("GenAI parsing failed. Switching to rule-based fallback parser...")
        # Pass the loaded SKILL_MAP to the fallback function
        parsed_data = parse_resume_with_rules(text, skill_map)
        parser_used = "Fallback (Rules)"
    else:
        parser_used = "Primary (GenAI)"

    print(f"Parsing complete. Method used: {parser_used}")
    return parsed_data, parser_used



### 7) Final Execution

In [ ]:
# 1. First, build the map
SKILL_MAP = build_skill_map_from_csv('skills.csv')
nlp = spacy.load('en_core_web_sm')

# 2. Then, process a file by soecifying its path
## Modify the line below and provide your own file path
sample_pdf_path = '~/sample_resume4.pdf'
if os.path.exists(sample_pdf_path) and SKILL_MAP:
    final_data, method = process_resume(sample_pdf_path, SKILL_MAP)
    print("\n--- Final Parsed Data ---")
    print(json.dumps(final_data, indent=2))

Successfully loaded skills.csv with 13893 rows.


/var/folders/6f/bc2q7h4x0xd_1v2gtlr1f7xc0000gp/T/ipykernel_96432/1537022553.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['altLabels'].fillna('', inplace=True)


Built a skill map with 27178 total variations.

--- Processing sample_resume4.pdf ---
Attempting to parse with GenAI...
Parsing complete. Method used: Primary (GenAI)

--- Final Parsed Data ---
{
  "name": "JULIE MONROE",
  "email": "email@email.com",
  "skills": [
    "Food preparation",
    "Kitchen maintenance",
    "Kitchen equipment operation",
    "Food sanitation",
    "Nutrition",
    "nutrition education counseling",
    "nutrition assessments",
    "WIC certifications",
    "nutritional status assessment",
    "behavior and lifestyle changes",
    "osteodystrophy management",
    "anemia management",
    "nutrition management",
    "bone mineral status monitoring",
    "TPN usage reduction",
    "nutritional care",
    "staff supervision",
    "mass media outreach",
    "program development"
  ]
}
